In [1]:
from glob import glob
from collections import defaultdict, Counter

import PIL
import PIL.Image
import numpy as np
PIL.Image.MAX_IMAGE_PIXELS = 3000000000

import os.path
from tqdm import tqdm

In [2]:
TILE_SIZE = 256

In [3]:
data_files = sorted( [ x.replace( 'data_tmp/', '' ) for x in glob( 'data_tmp/*' ) ] )
print( data_files[:2] )
all_genes = Counter()

#split by tissue
tmp = defaultdict( lambda: [] )
for i in data_files:
    if 'MAP' in i:
        continue
    i = i.split('.GENE_')
    tmp[ i[0] ].append(  i[1].replace( '.jpg', '' ) )
data_files = dict( tmp )

for key in data_files.keys():
    new = defaultdict( lambda:[] )
    for i in data_files[key]:
        i = i.split('.SLICE_')
        new[i[0]].append( int(i[1]) )
        all_genes.update( [ i[0] ] )
    data_files[key] = dict( new )
    

['W1-1-2-A.1.01.GENE_BIRC5.SLICE_4.MAP.jpg', 'W1-1-2-A.1.01.GENE_BIRC5.SLICE_4.jpg']


In [4]:
all_genes.most_common(5)

[('H+E', 11368), ('TGFBR2', 631), ('MET', 624), ('CD44', 608), ('HIF1A', 608)]

In [5]:
def make_file_list( t_list ):
    # 'W1-1-2-A.1.01.GENE_BIRC5.SLICE_4.MAP.jpg'
    out = []
    
    target_root = '{}.GENE_{}.SLICE_{}'.format(t_list[0], t_list[1], str(t_list[2]))
    out.append( target_root+'.jpg' )
    out.append( target_root+'.MAP.jpg' )
    
    HE_root = '{}.GENE_{}.SLICE_{}'.format(t_list[0], t_list[3], str(t_list[4]))
    out.append( HE_root+'.jpg' )
    out.append( HE_root+'.MAP.jpg' )
    
    for f in out:
        if not os.path.isfile('data_tmp/'+f):
            return None
    
    return out

In [6]:
def make_target_list( target_name ):
    '''returns list of pairs (target+HE) images that need
    to be analyzed for a given gene'''
    
    out = []
    
    for sample in data_files.keys(): #iterate over all samples
        if target_name not in data_files[sample] or not len(data_files[sample][target_name]):
            continue
        if 'H+E' not in data_files[sample] or not len(data_files[sample]['H+E']):
            continue
        
        target_slides = data_files[sample][target_name]
        HE_slides = data_files[sample]['H+E']
        
        for target_slide in target_slides:
            HE_match = min( (abs(target_slide-x),x) for x in HE_slides )[1]
            f_list = make_file_list( (sample, target_name, target_slide, 'H+E', HE_match) )
            if f_list:
                out.append( f_list )
    
    return out

In [33]:
def read_image( f ):
    f = 'data_tmp/' + f
    return np.array( PIL.Image.open(f) )

def cut_image( t, tm, h, hm, fname="test.jpg" ):
    OUT_NUM = 1
    t  = read_image( t  )
    tm = read_image( tm )
    h  = read_image( h  )
    hm = read_image( hm )

    print( t.shape, tm.shape, h.shape, hm.shape )
    
    for dx in range(0, t.shape[0], TILE_SIZE):
        for dy in range(0, t.shape[1], TILE_SIZE):
            t_tile = t[dx:dx+TILE_SIZE,dy:dy+TILE_SIZE,:]
            tm_tile = tm[dx:dx+TILE_SIZE,dy:dy+TILE_SIZE,:]
            h_tile = h[dx:dx+TILE_SIZE,dy:dy+TILE_SIZE,:]
            hm_tile = hm[dx:dx+TILE_SIZE,dy:dy+TILE_SIZE,:]
            
            #print( t_tile.shape, tm_tile.shape, h_tile.shape, hm_tile.shape)
            
            if not (t_tile.shape[0]==tm_tile.shape[0]==h_tile.shape[0]==hm_tile.shape[0]==TILE_SIZE):
                continue
            if not (t_tile.shape[1]==tm_tile.shape[1]==h_tile.shape[1]==hm_tile.shape[1]==TILE_SIZE):
                continue
                
            #check map for valid tile (right now just checking for white pixels)
            '''is_white = False
            for x in range( TILE_SIZE ):
                for y in range( TILE_SIZE ):
                    if all( x==255 for x in tm_tile[x,y,:] ):
                        is_white = True
                    if all( x==0 for x in tm_tile[x,y,:] ):
                        is_white = True
            if is_white:
                continue'''
            
            if( np.any( np.all( tm_tile==0, axis=0 ) ) ):
                continue
            if( np.any( np.all( tm_tile==255, axis=0 ) ) ):
                continue
                
            #output stiched image tile (y,x,c)
            out = np.zeros( shape=(TILE_SIZE,TILE_SIZE*2,3), dtype=np.uint8 )
            out[:,0:TILE_SIZE,:] = t_tile[:,:,:]
            out[:,TILE_SIZE:,:] = h_tile[:,:,:]
            
            #save image
            img = PIL.Image.fromarray(out)
            img.save(fname.replace('XXX', str(OUT_NUM)), "JPEG", quality=100, optimize=True, progressive=True)
            OUT_NUM += 1
            del img
            
    del t, tm, h, hm

In [35]:
gene_name = 'MET'

for i,(t, tm, h, hm) in tqdm( enumerate( make_target_list(gene_name) ) ):
    cut_image( t, tm, h, hm, fname="sliced_out/{}/{}_XXX.jpg".format(gene_name,i+1) )

0it [00:00, ?it/s]

(18080, 15040, 3) (18080, 15040, 3) (18080, 15040, 3) (18080, 15040, 3)


1it [01:30, 90.81s/it]

(18080, 15040, 3) (18080, 15040, 3) (18080, 15040, 3) (18080, 15040, 3)


2it [03:01, 90.87s/it]

...

452it [12:25:53, 98.47s/it]

(18176, 15104, 3) (18176, 15104, 3) (18080, 15040, 3) (18080, 15040, 3)


453it [12:27:36, 99.82s/it]

(18176, 15104, 3) (18176, 15104, 3) (18176, 15104, 3) (18176, 15104, 3)


KeyboardInterrupt: 